# COVID-19 en Chile
> Total de casos confirmados, fallecidos confirmados, pacientes en UCI por región.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/chart-preview.png

In [1]:
#hide
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
plt.style.use('ggplot')

In [407]:
#hide
from IPython.display import display, HTML

In [373]:
#hide
update_date = pd.to_datetime('today') - pd.offsets.Hour(19)
today = update_date.strftime('%Y-%m-%d')
today

'2020-04-21'

In [374]:
#hide
date_one_week_ago = (update_date - pd.offsets.Day(7)).strftime('%Y-%m-%d')
date_one_week_ago

'2020-04-14'

In [375]:
#hide_input
print(f"Última actualización: {update_date.strftime('%d/%m/%Y')}.")

Última actualización: 21/04/2020.


In [376]:
#hide
casos_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [419]:
#hide
s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" \
+ today + "-CasosConfirmados-totalRegional.csv"
deaths_raw = pd.read_csv(s, index_col='Region')

In [378]:
#hide
s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" \
+ date_one_week_ago + "-CasosConfirmados-totalRegional.csv"
deaths_one_week_ago = pd.read_csv(s, index_col='Region')

In [437]:
#hide
deaths_table = pd.DataFrame()
deaths_table["Region"] = deaths_raw["Fallecidos"].sort_values(ascending=False)[1:6].reset_index()["Region"]
deaths_table["Fallecidos confirmados"] = deaths_raw["Fallecidos"].sort_values(ascending=False)[1:6].reset_index()["Fallecidos"]

In [438]:
#hide
cases_table = pd.DataFrame()
cases_table['Region'] = casos_raw['2020-04-21'].sort_values(ascending=False)[1:6].reset_index()['Region']
cases_table['Casos confirmados'] = casos_raw['2020-04-21'].sort_values(ascending=False)[1:6].reset_index()[today]

In [436]:
#hide_input
format_dict = {'Region':'{}','Casos confirmados':'{}'}
(cases_table.style.format(format_dict)
  .hide_index()).highlight_max('Casos confirmados', color='lightgreen')

Region,Casos confirmados
Metropolitana,5788
Araucanía,1092
Ñuble,694
Biobío,636
Magallanes,593


In [435]:
#hide_input
format_dic = {'Region':'{}', 'Fallecimientos confirmados':'{}'}
(deaths_table.style.format(format_dict)
  .hide_index()).highlight_max('Fallecidos confirmados', color='lightgreen')

Region,Fallecidos confirmados
Metropolitana,69
Araucania,25
Nuble,13
Maule,10
Magallanes,7


In [379]:
#hide
casos_raw.iloc[-1,-1], deaths_raw.iloc[-1,-1]

(10832, 147)

In [380]:
#hide
total_confirmed = casos_raw.iloc[-1,-1]
total_deaths = deaths_raw.iloc[-1,-1]

In [381]:
#hide
total_confirmed_one_week_ago = casos_raw.iloc[-1,-8]
total_deaths_one_week_ago = deaths_one_week_ago.loc["Total", "Fallecidos"]

In [382]:
#hide
diff_cases = total_confirmed-total_confirmed_one_week_ago
diff_deaths = total_deaths-total_deaths_one_week_ago

In [383]:
#hide
total_confirmed = '{:,}'.format(total_confirmed).replace(',', '.')
total_deaths = '{:,}'.format(total_deaths).replace(',', '.')

In [384]:
#hide
diff_cases = '{:,}'.format(diff_cases).replace(',', '.')
diff_deaths = '{:,}'.format(diff_deaths).replace(',', '.')

In [385]:
#hide_input
HTML(f'<h2>Estadísticas para Chile ({update_date.strftime("%d/%m/%Y")}): </h2><p style="color:#3361ff;"><span style="font-weight:bold;">Casos confirmados:</span> {total_confirmed} (+{diff_cases} en una semana)</p><p style="color:#FF3F3F;"><span style="font-weight:bold;">Fallecimientos confirmados:</span> {total_deaths} (+{diff_deaths} en una semana)</p>')

# Evolución de casos confirmados por región

In [7]:
#hide
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [8]:
#hide
data_raw = data_raw.drop("Total")

In [9]:
#hide
data = pd.DataFrame()
i = 0
for date in data_raw.keys():
    for n, region in enumerate(data_raw.index):
        data[i] = date, region, data_raw[date].loc[region], n
        i += 1

In [10]:
#hide
data_raw = data_raw.reset_index()
regiones = data_raw['Region'].values
data = data.T
data = data.rename(columns={0: "date", 1: "region", 2: "casos", 3: "codigo region"})

In [11]:
#hide
data["casos"] = data["casos"].astype(int)

In [12]:
#hide_input
selection = alt.selection_multi(fields=['casos'], bind='legend')

bars = alt.Chart(data).mark_bar().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y=alt.Y('casos', axis=alt.Axis(title='Casos confirmados')),
    color=alt.Color('region:N', sort=regiones),
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    ),
    ).add_selection(
    selection
).properties(
    title='COVID-19 en Chile: Total de casos confirmados por región'
)
    
# text = bars.mark_text(
#     align='left',
#     baseline='middle',
#     dx=3  # Nudges text to right so it doesn't appear on top of the bar
# ).encode(
#     text='sum(pacientes):Q'
# )

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

bars.properties(width=600, height=300)
# (bars + text).properties(width=600, height=300)

alt.Chart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

Inspirado del [tweet](https://twitter.com/NachoToledoR/status/1244631395781218306) de [@NachoToledoR](https://twitter.com/NachoToledoR).

# Evolución de fallecidos confirmados por región

In [13]:
#hide
data = pd.DataFrame()
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/alonsosilvaallende/Datos-COVID19/master/output/producto4/2020-03-24-CasosConfirmados-totalRegional.csv",
    index_col='Region')
data['2020-03-24'] = data_raw['Fallecidos']

In [14]:
#hide
first_death_date = '2020-03-24'
total_days = (pd.to_datetime(today)-pd.to_datetime(first_death_date)).days

In [15]:
#hide
for i in np.arange(total_days+1):
  date = (pd.to_datetime(first_death_date)+pd.DateOffset(i)).strftime('%Y-%m-%d')
  s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" + date + "-CasosConfirmados-totalRegional.csv"
  data_by_date = pd.read_csv(s)
  if 'Fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Fallecidos"].values
  elif 'Casos fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Casos fallecidos"].values
  else:
    data[date] = data_by_date[" Casos fallecidos"].values

In [16]:
#hide
data = data.drop("Total")

In [17]:
#hide
new_data = pd.DataFrame()
i = 0
for date in data.keys():
    for n, region in enumerate(data.index):
        new_data[i] = date, region, data[date].loc[region], n
        i += 1

In [18]:
#hide
data = data.reset_index()
regiones = data['Region'].values
new_data = new_data.T
new_data = new_data.rename(columns={0: "date", 1: "region", 2: "fallecidos", 3: "codigo region"})

In [19]:
#hide
new_data["fallecidos"] = new_data["fallecidos"].astype(int)

In [20]:
#hide_input
bars = alt.Chart(new_data).mark_bar().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y=alt.Y('fallecidos', axis=alt.Axis(title='Fallecidos confirmados')),
    color=alt.Color('region:N', sort=regiones),
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    )
    ).properties(
    title='COVID-19 en Chile: Total de fallecidos confirmados por región'
)
    
# text = bars.mark_text(
#     align='left',
#     baseline='middle',
#     dx=3  # Nudges text to right so it doesn't appear on top of the bar
# ).encode(
#     text='sum(pacientes):Q'
# )

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

bars.properties(width=600, height=300)
# (bars + text).properties(width=600, height=300)

alt.Chart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Evolución de pacientes en UCI por región

In [21]:
#hide
data_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto8/UCI.csv',
    index_col='Region')
data_raw = data_raw.drop(columns='Codigo region')
data_raw.loc['Total'] = data_raw.sum(axis=0)
data_raw = data_raw.drop(columns='Poblacion')

In [22]:
#hide
data_raw = data_raw.drop("Total")

In [23]:
#hide
data = pd.DataFrame()
i = 0
for date in data_raw.keys():
    for n, region in enumerate(data_raw.index):
        data[i] = date, region, data_raw[date].loc[region], n
        i += 1

In [24]:
#hide
data_raw = data_raw.reset_index()

In [25]:
#hide
regiones = data_raw['Region'].values

In [26]:
#hide
data = data.T

In [27]:
#hide
data = data.rename(columns={0: "date", 1: "region", 2: "pacientes", 3: "codigo region"})

In [28]:
#hide
data["pacientes"] = data["pacientes"].astype(int)

In [29]:
#hide_input
bars = alt.Chart(data).mark_bar().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y=alt.Y('pacientes'),
    color=alt.Color('region:N', sort=regiones),
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    )
    ).properties(
    title='COVID-19 en Chile: Pacientes en UCI por región'
)
    
# text = bars.mark_text(
#     align='left',
#     baseline='middle',
#     dx=3  # Nudges text to right so it doesn't appear on top of the bar
# ).encode(
#     text='sum(pacientes):Q'
# )

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

bars.properties(width=600, height=300)
# (bars + text).properties(width=600, height=300)

alt.Chart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Casos confirmados por región

In [30]:
#hide
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [31]:
#hide
data_raw = data_raw.drop('Total')
data_raw = data_raw.reset_index()

In [32]:
#hide
regiones = data_raw["Region"].values
regiones

array(['Arica y Parinacota', 'Tarapacá', 'Antofagasta', 'Atacama',
       'Coquimbo', 'Valparaíso', 'Metropolitana', 'O’Higgins', 'Maule',
       'Ñuble', 'Biobío', 'Araucanía', 'Los Ríos', 'Los Lagos', 'Aysén',
       'Magallanes'], dtype=object)

In [33]:
#hide
data_raw = data_raw[['Region', today]]

In [34]:
#hide_input
bars = alt.Chart(data_raw).mark_bar().encode(
    x = alt.X(today, scale=alt.Scale(domain=(0, 5600)), axis=alt.Axis(title='Casos confirmados')),
    y = alt.Y('Region:N', sort='-x')
).properties(
    title='COVID-19 en Chile: Casos confirmados por región'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=today
)

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Casos confirmados por región por cada 100.000 habitantes

In [35]:
#hide
tests_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto7/PCR.csv',
    index_col='Region')

In [36]:
#hide
data_raw["Tasa de casos"] = 100000*data_raw[today]/tests_raw["Poblacion"].values

In [37]:
#hide_input
bars = alt.Chart(data_raw).mark_bar().encode(
    x = alt.X('Tasa de casos:Q', scale=alt.Scale(domain=(0, 340)), axis=alt.Axis(title='Casos confirmados por cada 100.000 habitantes')),
    y = alt.Y('Region:N', sort='-x')
).properties(
    title='COVID-19 en Chile: Casos confirmados por región por cada 100.000 habitantes'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Tasa de casos:Q', format=',.2f')
)

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Fallecidos confirmados por región

In [38]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/' + today +'-CasosConfirmados-totalRegional.csv'
data_raw = pd.read_csv(s, index_col='Region')

In [39]:
#hide
data_raw = data_raw[data_raw['Fallecidos']>0]
data_raw = data_raw.drop('Total')

In [40]:
#hide_input
bars = alt.Chart(data_raw.reset_index()).mark_bar().encode(
    x = alt.X('Fallecidos:Q', scale=alt.Scale(domain=(0, 70)), axis=alt.Axis(title='Fallecidos confirmados')),
    y = alt.Y('Region:N', sort='-x')
).properties(
    title='COVID-19 en Chile: Fallecidos confirmados por región'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Fallecidos:Q'
)

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Fallecidos confirmados por región por cada 100.000 habitantes

In [41]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/' + today + '-CasosConfirmados-totalRegional.csv'
data_raw = pd.read_csv(s, index_col='Region')

In [42]:
#hide
tests_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto7/PCR.csv',
    index_col='Region')
tests_raw = tests_raw.fillna(0)

In [43]:
#hide
tests_total = pd.Series(dtype='float64')
for date in tests_raw.columns:
  tests_raw[date] = tests_raw[date].apply(lambda x: 0 if x=='-' else x)
  tests_raw[date] = tests_raw[date].astype(int)
  tests_total[date] = np.sum(tests_raw[date])

In [44]:
#hide
tests_raw.loc['Total'] = tests_total

In [45]:
#hide
death_rate = 100000*data_raw['Fallecidos']/tests_raw['Poblacion'].values
death_rate = death_rate.sort_values(ascending=True)
death_rate = death_rate[death_rate>0]

In [46]:
#hide_input
bars = alt.Chart(death_rate.reset_index()).mark_bar().encode(
    x = alt.X('Fallecidos:Q', scale=alt.Scale(domain=(0, 3.6)), axis=alt.Axis(title='Fallecidos confirmados por cada 100.000 habitantes')),
    y = alt.Y('Region:N', sort='-x')
).properties(
    title='COVID-19 en Chile: Fallecidos confirmados por región por cada 100.000 habitantes'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Fallecidos:Q', format=',.2f')
)

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Tasa de letalidad por región
> Tasa de letalidad = fallecidos confirmados / casos confirmados (en porcentaje)

In [47]:
#hide
CFR = data_raw['Fallecidos']/data_raw['Casos  totales']*100
CFR = CFR.sort_values(ascending=True)
CFR = CFR[CFR>0]

In [48]:
#hide
CFR = CFR.reset_index()
CFR['Tasa de letalidad'] = CFR[0]
CFR = CFR.drop(columns=0)

In [49]:
#hide_input
bars = alt.Chart(CFR).mark_bar().encode(
    x = alt.X('Tasa de letalidad:Q', axis=alt.Axis(title='Tasa de letalidad')),
    y = alt.Y('Region:N', sort='-x')
).properties(
    title='COVID-19 en Chile: Tasa de letalidad por región'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Tasa de letalidad:Q', format=',.1f')
)

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Exámenes informados por región desde el 9 de abril
Nota: sólo se dispone de los datos de exámenes informados por región desde el 9 de abril

In [50]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-04-08-CasosConfirmados-totalRegional.csv'
data_20200408 = pd.read_csv(s, index_col='Region')

In [51]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/' + today +'-CasosConfirmados-totalRegional.csv'
data_update_date = pd.read_csv(s, index_col='Region')

In [52]:
#hide
tests_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto7/PCR.csv',
    index_col='Region')
tests_raw = tests_raw.fillna(0)

In [53]:
#hide
tests_total = pd.Series(dtype='float64')
for date in tests_raw.columns:
  tests_raw[date] = tests_raw[date].apply(lambda x: 0 if x=='-' else x)
  tests_raw[date] = tests_raw[date].astype(int)
  tests_total[date] = np.sum(tests_raw[date])

In [54]:
#hide
tests_raw.loc['Total'] = tests_total

In [55]:
#hide
data = pd.DataFrame()
data["Fallecidos"] = data_update_date["Fallecidos"]-data_20200408["Fallecidos"]
data["Casos"] = data_update_date["Casos  totales"]-data_20200408["Casos  totales"]
data["Tests"] = tests_raw.drop(columns='Poblacion').sum(axis=1).values

In [56]:
#hide
data_per_capita = pd.DataFrame()
data_per_capita["Fallecidos"] = 100000*data["Fallecidos"]/tests_raw["Poblacion"].values
data_per_capita["Casos"] = 100000*data["Casos"]/tests_raw["Poblacion"].values
data_per_capita["Tests"] = 100000*data["Tests"]/tests_raw["Poblacion"].values

In [57]:
#hide
casos_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv", 
    index_col='Region')

In [58]:
#hide
casos_20200408 = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-04-08-CasosConfirmados-totalRegional.csv', index_col='Region')

In [59]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/' + today + '-CasosConfirmados-totalRegional.csv'
casos_update_date = pd.read_csv(s, index_col='Region')

In [60]:
#hide
positividad = 100*(casos_update_date['Casos  totales'] - casos_20200408['Casos  totales'])/tests_raw.drop(columns='Poblacion').sum(axis=1).values

In [61]:
#hide
tests_raw['tests informados'] = tests_raw.drop(columns='Poblacion').sum(axis=1)

In [62]:
#hide
tests_raw['tasa de tests'] = 100000*tests_raw['tests informados']/tests_raw['Poblacion']

In [63]:
#hide_input
bars = alt.Chart(tests_raw.drop('Total').reset_index()).mark_bar().encode(
    x = alt.X('tests informados:Q', axis=alt.Axis(title='Exámenes informados')),
    y = alt.Y('Region:N', sort='-x')
).properties(
    title='COVID-19 en Chile: Total de exámenes informados por región desde el 9 de abril'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('tests informados:Q', format='.0f')
)

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Exámenes informados por región por cada 100.000 habitantes desde el 9 de abril
Nota: sólo se dispone de los datos de exámenes informados por región desde el 9 de abril

In [64]:
#hide_input
bars = alt.Chart(tests_raw.drop('Total').reset_index()).mark_bar().encode(
    x = alt.X('tasa de tests:Q', axis=alt.Axis(title='Exámenes informados')),
    y = alt.Y('Region:N', sort='-x')
).properties(
    title='COVID-19 en Chile: Exámenes informados por región por cada 100.000 habitantes desde el 9 de abril'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('tasa de tests:Q', format='.1f')
)

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Exámenes, casos confirmados, y fallecimientos confirmados por región  desde el 9 de abril
Nota: sólo se dispone de los datos de exámenes informados por región desde el 9 de abril

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por región desde el 9 de abril.

In [65]:
#hide
data = data.sort_values(by="Tests", ascending=False)
data_per_capita = data_per_capita.sort_values(by="Tests", ascending=False)

In [66]:
#hide
data["codigo"]=np.arange(len(data))
data_per_capita["codigo"]=np.arange(len(data))

In [67]:
#hide_input
bar = alt.Chart(data.drop("Total").reset_index()).mark_bar(opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title='Exámenes informados, casos confirmados, fallecimientos confirmados')),
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
).properties(
    title='COVID-19 en Chile: Exámenes informados, casos confirmados y fallecidos confirmados por región',
    width=alt.Step(40)  # controls width of bar.
)

tick0 = alt.Chart(data.drop("Total").reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=40 * 0.4,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

tick1 = alt.Chart(data.drop("Total").reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=40 * 0.4,  # controls width of tick.
).encode(
    x='Casos:Q',
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

tick2 = alt.Chart(data.drop("Total").reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=40 * 0.4,  # controls width of tick.
).encode(
    x='Fallecidos:Q',
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

(bar + tick0 + tick1 + tick2).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Exámenes, casos confirmados, y fallecimientos confirmados por región por cada 100.000 habitantes desde el 9 de abril
Nota: sólo se dispone de los datos de exámenes informados por región desde el 9 de abril

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por región por cada 100.000 habitantes desde el 9 de abril

In [70]:
#hide_input
bar = alt.Chart(data_per_capita.drop("Total").reset_index()).mark_bar(opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title='Exámenes informados, casos confirmados, fallecimientos confirmados')),
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
).properties(
    title='COVID-19 en Chile: Exámenes informados, casos confirmados y fallecidos confirmados por región por cada 100.000 habitantes',
    width=alt.Step(40)  # controls width of bar.
)

tick0 = alt.Chart(data_per_capita.drop("Total").reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('Region', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

tick1 = alt.Chart(data_per_capita.drop("Total").reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Casos:Q',
    y=alt.Y('Region', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

tick2 = alt.Chart(data_per_capita.drop("Total").reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Fallecidos:Q',
    y=alt.Y('Region', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

(bar + tick0 + tick1 + tick2).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Exámenes, casos confirmados, y fallecimientos confirmados por país de la OCDE por cada 1 millón de habitantes

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por país por cada 1 millón de habitantes

In [176]:
#hide
test_data_raw = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/testing/covid-testing-all-observations.csv')

In [237]:
#hide
list_OCDE = ['Australia', 'Austria', 'Belgium', 'Canada', 'Chile', 
                  'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
                  'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland',
                  'Israel', 'Italy', 'Japan', 'Korea', 'Latvia',
                  'Lithuania', 'Luxembourg', 'Mexico', 'Netherlands', 'New Zealand',
                  'Norway', 'Poland', 'Portugal', 'Slovakia', 'Slovenia',
                  'Spain', 'Sweden', 'Switzerland', 'Turkey', 'United Kingdom']

In [238]:
#hide
test_data = pd.DataFrame()
for country in list_OCDE:
    test_data[country] = test_data_raw[test_data_raw['Entity'].str.contains(country)].iloc[-1]

In [239]:
#hide
# United States tiene dos series. Utilizo la que tiene datos más recientes.
test_data['United States'] = test_data_raw[test_data_raw['Entity'].str.contains('United States - inconsistent units')].iloc[-1]

In [240]:
#hide
test_data = test_data.T

In [241]:
#hide
data = pd.DataFrame()
data['Tests'] = test_data['Cumulative total']

In [242]:
#hide
list_OCDE = list_OCDE + ['United States']

In [183]:
#hide
full_data = pd.read_csv("https://covid.ourworldindata.org/data/ecdc/full_data.csv", index_col='date')

In [184]:
#hide
cases = pd.Series()
deaths = pd.Series()
for country in list_OCDE:
    cases[country] = full_data[full_data['location'].str.contains(country)].iloc[-1]['total_cases']
    deaths[country] = full_data[full_data['location'].str.contains(country)].iloc[-1]['total_deaths']

In [185]:
#hide
data['Cases'] = cases
data['Deaths'] = deaths

In [186]:
#hide
locations_data = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/locations.csv')

In [187]:
#hide
population = pd.Series()
for country in list_OCDE:
    population[country] = locations_data[locations_data['location'].str.contains(country)].iloc[-1]['population']

In [188]:
#hide
data['Population'] = population

In [189]:
#hide
data['Tests'] = 1000000*data['Tests']/data['Population']
data['Cases'] = 1000000*data['Cases']/data['Population']
data['Deaths'] = 1000000*data['Deaths']/data['Population']

In [190]:
#hide
data = data.sort_values(by='Tests', ascending=False)
data['codigo'] = np.arange(len(data))

In [192]:
#hide_input
bar = alt.Chart(data.reset_index()).mark_bar(opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title='Exámenes informados, casos confirmados, fallecimientos confirmados por cada 1 millón de habitantes')),
    y=alt.Y('index:N', axis=alt.Axis(title='País'), sort=alt.EncodingSortField(field='codigo', order='ascending'))
).properties(
    title='COVID-19: Exámenes informados, casos confirmados y fallecidos confirmados por país por cada 1 millón de habitantes',
    width=alt.Step(40)  # controls width of bar.
)

tick0 = alt.Chart(data.reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

tick1 = alt.Chart(data.reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Cases:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

tick2 = alt.Chart(data.reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Deaths:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

(bar + tick0 + tick1 + tick2)

alt.LayerChart(...)

Fuentes: ECDC, [Our World in Data](https://ourworldindata.org/)

# Exámenes, casos confirmados, y fallecimientos confirmados por país de la OCDE por cada 1 millón de habitantes en los últimos 3 días

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por país por cada 1 millón de habitantes

In [270]:
#hide
data = pd.DataFrame()
data['Tests'] = test_data['3-day rolling mean daily change']

In [271]:
#hide
cases = pd.Series()
deaths = pd.Series()
for country in list_OCDE:
    cases[country] = full_data[full_data['location'].str.contains(country)].iloc[-3:]['new_cases'].mean()
    deaths[country] = full_data[full_data['location'].str.contains(country)].iloc[-3:]['new_deaths'].mean()

In [272]:
#hide
data['Cases'] = cases
data['Deaths'] = deaths

In [273]:
#hide
population = pd.Series()
for country in list_OCDE:
    population[country] = locations_data[locations_data['location'].str.contains(country)].iloc[-1]['population']

In [274]:
#hide
data['Population'] = population

In [275]:
#hide
data['Tests'] = 1000000*data['Tests']/data['Population']
data['Cases'] = 1000000*data['Cases']/data['Population']
data['Deaths'] = 1000000*data['Deaths']/data['Population']

In [282]:
#hide
data = data.dropna()

In [277]:
#hide
data = data.sort_values(by='Tests', ascending=False)
data['codigo'] = np.arange(len(data))

In [281]:
#hide_input
bar = alt.Chart(data.reset_index()).mark_bar(opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title=f'Exámenes informados, casos confirmados, fallecimientos confirmados por cada 1 millón de habitantes\n(promedio de los últimos 3 días)')),
    y=alt.Y('index:N', axis=alt.Axis(title='País'), sort=alt.EncodingSortField(field='codigo', order='ascending'))
).properties(
    title=f'COVID-19: Exámenes informados, casos confirmados y fallecidos confirmados por país por cada 1 millón de habitantes\n(promedio de los últimos 3 días)',
    width=alt.Step(40)  # controls width of bar.
)

tick0 = alt.Chart(data.reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

tick1 = alt.Chart(data.reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Cases:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

tick2 = alt.Chart(data.reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Deaths:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

(bar + tick0 + tick1 + tick2)

alt.LayerChart(...)

# Exámenes, casos confirmados, y fallecimientos confirmados por país de latinoamérica por cada 1 millón de habitantes

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por país por cada 1 millón de habitantes

Nota: Muchos países de latinoamérica no tienen datos disponibles.

In [283]:
#hide
list_latinoamerica = ['Argentina', 'Bolivia', 'Chile', 'Colombia',
                     'Costa Rica', 'Cuba', 'Ecuador', 'El Salvador',
                     'Mexico', 'Panama', 'Paraguay',
                     'Peru', 'Uruguay']

In [284]:
#hide
test_data = pd.DataFrame()
for country in list_latinoamerica:
    test_data[country] = test_data_raw[test_data_raw['Entity'].str.contains(country)].iloc[-1]

In [285]:
#hide
test_data = test_data.T

In [286]:
#hide
data = pd.DataFrame()
data['Tests'] = test_data['Cumulative total']

In [287]:
#hide
cases = pd.Series()
deaths = pd.Series()
for country in list_latinoamerica:
    cases[country] = full_data[full_data['location'].str.contains(country)].iloc[-1]['total_cases']
    deaths[country] = full_data[full_data['location'].str.contains(country)].iloc[-1]['total_deaths']

In [288]:
#hide
data['Cases'] = cases
data['Deaths'] = deaths

In [289]:
#hide
population = pd.Series()
for country in list_latinoamerica:
    population[country] = locations_data[locations_data['location'].str.contains(country)].iloc[-1]['population']

In [290]:
#hide
data['Population'] = population

In [291]:
#hide
data['Tests'] = 1000000*data['Tests']/data['Population']
data['Cases'] = 1000000*data['Cases']/data['Population']
data['Deaths'] = 1000000*data['Deaths']/data['Population']

In [292]:
#hide
data = data.sort_values(by='Tests', ascending=False)
data['codigo'] = np.arange(len(data))

In [293]:
#hide_input
bar = alt.Chart(data.reset_index()).mark_bar(opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title='Exámenes informados, casos confirmados, fallecimientos confirmados por cada 1 millón de habitantes')),
    y=alt.Y('index:N', axis=alt.Axis(title='País'), sort=alt.EncodingSortField(field='codigo', order='ascending'))
).properties(
    title='COVID-19: Exámenes informados, casos confirmados y fallecidos confirmados por país por cada 1 millón de habitantes',
    width=alt.Step(40)  # controls width of bar.
)

tick0 = alt.Chart(data.reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

tick1 = alt.Chart(data.reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Cases:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

tick2 = alt.Chart(data.reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Deaths:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

(bar + tick0 + tick1 + tick2)

alt.LayerChart(...)

Fuentes: ECDC, [Our World in Data](https://ourworldindata.org/)

# Exámenes, casos confirmados, y fallecimientos confirmados por país de latinoamérica por cada 1 millón de habitantes en los últimos 3 días

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por país por cada 1 millón de habitantes

Nota: Muchos países de latinoamérica no tienen datos disponibles.

In [294]:
#hide
data = pd.DataFrame()
data['Tests'] = test_data['3-day rolling mean daily change']

In [295]:
#hide
cases = pd.Series()
deaths = pd.Series()
for country in list_latinoamerica:
    cases[country] = full_data[full_data['location'].str.contains(country)].iloc[-3:]['new_cases'].mean()
    deaths[country] = full_data[full_data['location'].str.contains(country)].iloc[-3:]['new_deaths'].mean()

In [296]:
#hide
data['Cases'] = cases
data['Deaths'] = deaths

In [297]:
#hide
population = pd.Series()
for country in list_latinoamerica:
    population[country] = locations_data[locations_data['location'].str.contains(country)].iloc[-1]['population']

In [298]:
#hide
data['Population'] = population

In [299]:
#hide
data['Tests'] = 1000000*data['Tests']/data['Population']
data['Cases'] = 1000000*data['Cases']/data['Population']
data['Deaths'] = 1000000*data['Deaths']/data['Population']

In [301]:
#hide
data = data.dropna()

In [302]:
#hide
data = data.sort_values(by='Tests', ascending=False)
data['codigo'] = np.arange(len(data))

In [303]:
#hide_input
bar = alt.Chart(data.reset_index()).mark_bar(opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title=f'Exámenes informados, casos confirmados, fallecimientos confirmados por cada 1 millón de habitantes\n(promedio de los últimos 3 días)')),
    y=alt.Y('index:N', axis=alt.Axis(title='País'), sort=alt.EncodingSortField(field='codigo', order='ascending'))
).properties(
    title=f'COVID-19: Exámenes informados, casos confirmados y fallecidos confirmados por país por cada 1 millón de habitantes\n(promedio de los últimos 3 días)',
    width=alt.Step(40)  # controls width of bar.
)

tick0 = alt.Chart(data.reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

tick1 = alt.Chart(data.reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Cases:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

tick2 = alt.Chart(data.reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Deaths:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

(bar + tick0 + tick1 + tick2)

alt.LayerChart(...)